In [1]:
# From: https://www.kaggle.com/c/dog-breed-identification/data
# Author: Morpheus Hsieh

from __future__ import print_function, division

import os, sys
import copy
import io
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir
from os.path import join, isfile, split, exists
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.4.0+cu121
Torchvision Version:  0.19.0+cu121


In [2]:
# Load parameters from json file

# OutPath = r'D:\GitWork\dog_breed\output'
OutPath = '/home/jovyan/output/dog-breed/'

# cfgPath = r'D:\GitWork\dog_breed\configs'
# fname = 'Params_20200923-2230.json'
# json_file = join(cfgPath, fname)

# with open(json_file) as fin: 
#     Params = json.load(fin) 

# Params = {
#     'DataPath'    : r'D:\GitWork\dog_breed\data',
#     'OutPath'     : r'D:\GitWork\dog_breed\output',
#     'ProcPath'    : r'D:\GitWork\dog_breed\processed',
#     'PreTrainPath': r'D:\GitWork\dog_breed\pretrained',
#     'TestPath'    : r'D:\Dataset\dog-breed-identification\test',
#     'TrainPath'   : r'D:\Dataset\dog-breed-identification\train',
#     'PreTrainFile': 'resnet50_20200926-2053_t9175_v9339.pth',
#     'CsvLabel'    : 'labels.csv',
#     'BatchSize'   : 16,
#     'FracForTrain': 0.8
# }

Params = {
    'DataPath'    : r'/home/jovyan/data/dog-breed/',
    'OutPath'     : r'/home/jovyan/output/dog-breed/',
    'ProcPath'    : r'/home/jovyan/output/dog-breed/',
    'PreTrainPath': r'/home/jovyan/models/dog-breed/',
    'PreTrainFile': r'resnet50_20240801-2306_acc88.pth',
    'TestPath'    : r'/home/jovyan/data/dog-breed/test',
    'TrainPath'   : r'/home/jovyan/data/dog-breed/train',
    'CsvLabel'    : 'labels.csv',
    'BatchSize'   : 16,
    'FracForTrain': 0.8
}

print('Parameters:')
print(json.dumps(Params, indent=4))

PreTranPath = Params['PreTrainPath']
PreTranFile = Params['PreTrainFile']
Pretrain_abspath = join(PreTranPath, PreTranFile)

if not exists(Pretrain_abspath) or not isfile(Pretrain_abspath):
    outstr = "\n'{}' model not found...".format(Pretrain_abspath)
    print(outstr)
    raise SystemExit(outstr)


Parameters:
{
    "DataPath": "/home/jovyan/data/dog-breed/",
    "OutPath": "/home/jovyan/output/dog-breed/",
    "ProcPath": "/home/jovyan/output/dog-breed/",
    "PreTrainPath": "/home/jovyan/models/dog-breed/",
    "PreTrainFile": "resnet50_20240801-2306_acc88.pth",
    "TestPath": "/home/jovyan/data/dog-breed/test",
    "TrainPath": "/home/jovyan/data/dog-breed/train",
    "CsvLabel": "labels.csv",
    "BatchSize": 16,
    "FracForTrain": 0.8
}


In [3]:
# Read labels information

DataPath = Params.get('DataPath')
csv_labels = Params.get('CsvLabel')
f_abspath = join(DataPath, csv_labels)

df_labels = pd.read_csv(f_abspath)

print(df_labels.info())
print(); print(df_labels.head())

NumClasses = df_labels.shape[0]
print('\nNum classes:', NumClasses)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10222 non-null  object
 1   breed   10222 non-null  object
dtypes: object(2)
memory usage: 159.8+ KB
None

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever

Num classes: 10222


In [4]:
# Count all breeds
def countBreeds(df):
    df1 = df_labels.groupby("breed")["id"].count().reset_index(name="count")
    df1 = df1.sort_values(by='count', ascending=False).reset_index(drop=True)
    df1.insert(0, 'breed_id', df1.index)
    return df1

df_breeds = countBreeds(df_labels)
print(df_breeds.info())
print(); print(df_breeds.head())

NumClasses = int(df_breeds.shape[0])
print('\nNum classes:', NumClasses)

selected_breeds = df_breeds['breed'].tolist()

# dict_bid_fw = dict(df_breeds[['breed', 'breed_id']].values)
dict_bid_bw = dict(df_breeds[['breed_id', 'breed']].values)

def prettyPrint(d, indent=0):
    print('{')
    for key, value in d.items():
        if isinstance(value, dict):
            print('  ' * indent + str(key))
            prettyPrint(value, indent+1)
        else:
            print('  ' * (indent+1) + f"{key}: {value}")
    print('}')
                
print('\nBreeds dict backward:'); 
prettyPrint(dict_bid_bw)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   breed_id  120 non-null    int64 
 1   breed     120 non-null    object
 2   count     120 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.9+ KB
None

   breed_id                 breed  count
0         0    scottish_deerhound    126
1         1           maltese_dog    117
2         2          afghan_hound    116
3         3           entlebucher    115
4         4  bernese_mountain_dog    114

Num classes: 120

Breeds dict backward:
{
  0: scottish_deerhound
  1: maltese_dog
  2: afghan_hound
  3: entlebucher
  4: bernese_mountain_dog
  5: shih-tzu
  6: great_pyrenees
  7: pomeranian
  8: basenji
  9: samoyed
  10: airedale
  11: tibetan_terrier
  12: leonberg
  13: cairn
  14: beagle
  15: japanese_spaniel
  16: australian_terrier
  17: blenheim_spaniel
  18: miniature_pinscher
  19: iris

In [5]:
# Build dataset

# Transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

class myDataset(Dataset):

    def __init__(self, path, transform=None):
        
        img_list = [
            f.replace('.jpg', '') for f in listdir(path) \
            if f.endswith('.jpg') and isfile(join(path, f))
        ]

        self.len = len(img_list)
        self.images = img_list
        self.transform = transform
        self.path = path

    def __getitem__(self, index):
        iid = self.images[index]
        img = join(self.path, iid) + '.jpg'
        img_pil = Image.open(img)

        if self.transform is not None:
            img_tensor = self.transform(img_pil)

        return [img_tensor, iid]

    def __len__(self):
        return self.len

    
TestPath = Params['TestPath']
# BatchSize = Params['BatchSize']
BatchSize = 100
    
dataSet = myDataset(TestPath, transform=transform)
dataLoader = DataLoader(dataSet, batch_size=BatchSize, shuffle=False)
dataSize = len(dataSet)

imgs, iids = next(iter(dataLoader))
print('\nImage shape:', imgs.shape)

print('\nImage ids')
id_list = [''.join(iid) for iid in iids]
print('  '+'\n  '.join(id_list))

img = imgs[0]
print('\nImage shape:', img.shape)

print('\nImage tensor:')
print(img)


Image shape: torch.Size([100, 3, 224, 224])

Image ids
  e458d6133dd1b436b047f86910f7fe34
  635e859fef2e4cfaf5546c4d0265c88e
  944ec36bc768ee309e51bf881dabcb15
  616bf51988ee33e53d0e96c2367594fb
  6c834ba0465f70e9abb2629864530b64
  6899f99568a71ffa7d0affe51b759c23
  e7afb35b8252782013a6700e28e327fa
  cb808a3e412ab8cb51e69ae07bd1d6fe
  07ad25df7e380e29aa4a5788a96cef73
  03205e3e568c87e1568a8415272a8da4
  114cdd56bf41af845a83404b1a57ab82
  76e0cd14e2485f3b87abd6b7363400e8
  d786274a5042d9f222833b8dd38d0550
  18dba93ad1e08a88aa83c3c8d58a5429
  19905a98817ec6df4765ad5713558a76
  769df36b2485b3e9df740ca8e9e49793
  48dfb3ec4a4247325478e1c33c0d1683
  85819cf427f6e9806a16b087a1622c5a
  0cb0cc50ad302ff64f3297ce361ec485
  3ada400595a24fb5ff3c1fab4dea1581
  291d224aac9198f9c314fed2d247b34e
  9e72af813948e3349bc6b3454b4a6e52
  84590d2967993bf557e99de3d20658cd
  4f7ec53fb020dfd90ac36227ab8233dc
  fa85d9951d7996c4c92ce5dc41d87dcd
  c364569a4a1eba0dd82f4ae244e60aeb
  85f5f54e24db0bb9b2a44e37f67ad60d

In [6]:
# Use GPU for train
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")
print(device)

cuda:0


In [7]:
# Build Model 
# model = models.resnet50(pretrained=True)
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# New final layer with NumClasses
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NumClasses)

# load pretrained mode
print('Load model:', Pretrain_abspath)
model.load_state_dict(torch.load(Pretrain_abspath, weights_only=True))

if use_gpu: model = model.cuda()

print(model)

Load model: /home/jovyan/models/dog-breed/resnet50_20240801-2306_acc88.pth
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample

In [8]:
# Pediction
import torch.nn.functional as nnf

# Output submission
# fname_submission = 'submission.csv'
# f_abspath = join(OutPath, fname_submission)

cols_preds = ['id', 'prediction']
df_preds = pd.DataFrame(columns=cols_preds)

cols_probs = ['id'] + selected_breeds
df_probs = pd.DataFrame(columns=cols_probs)

start_time = time.time()
print('Start testing...')

model.eval()

for i, (inputs, iids) in enumerate(dataLoader):

    inputs = Variable(inputs.cuda())
    iid_list = list(iids)
    
    # with torch.set_grad_enabled(True):
    with torch.no_grad():
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        
        if i == 0:
            # print(); print(len(iid_list)); print('\n'.join(iid_list))
            print('\nProbs:'); print(probs.shape); print(probs)
            print('\nPreds:'); print(preds.shape); print(preds)
            print()
    
    pred_list = preds.tolist()
    pred_breeds = [dict_bid_bw.get(x) for x in pred_list]
    
    df_tmp = pd.DataFrame({
        'id': iid_list,
        'prediction': pred_breeds
    })
    # df_preds = df_preds.append(df_tmp)
    df_preds = pd.concat([df_preds, df_tmp], ignore_index=True)

    df_tmp = pd.DataFrame({'id': iid_list})
    
    # df_tmp[selected_breeds] = pd.DataFrame(probs.tolist())
    probs_df = pd.DataFrame(probs.tolist(), columns=selected_breeds)
    df_tmp = pd.DataFrame(index=probs_df.index)
    df_tmp = pd.concat([df_tmp, probs_df], axis=1, ignore_index=True)
    df_tmp = df_tmp.copy()    

    # df_probs = df_probs.append(df_tmp)
    df_probs = pd.concat([df_probs, df_tmp], ignore_index=True)

    print(i, end=', ')
    
print()
print('Testing time: {:10f} minutes'.format((time.time()-start_time)/60))    

print(); print(df_preds.info())
print(); print(df_preds.head())

print(); print(df_probs.info())
print(); print(df_probs.head())

Start testing...

Probs:
torch.Size([100, 120])
tensor([[0.0087, 0.0067, 0.0100,  ..., 0.0079, 0.0093, 0.0066],
        [0.0079, 0.0072, 0.0105,  ..., 0.0094, 0.0091, 0.0070],
        [0.0046, 0.0083, 0.0077,  ..., 0.0045, 0.0054, 0.0067],
        ...,
        [0.0032, 0.0074, 0.0049,  ..., 0.0028, 0.0048, 0.0034],
        [0.0116, 0.0175, 0.0076,  ..., 0.0175, 0.0181, 0.0056],
        [0.0301, 0.0073, 0.0192,  ..., 0.0101, 0.0139, 0.0067]],
       device='cuda:0')

Preds:
torch.Size([100])
tensor([ 81, 110,   4,  47,   5,  42,  35,  11, 107,  60,   6,  72,  92,  14,
        107,  87,   2, 111,  21,  69,  97,  48,  29,  97, 108,  59,  21, 103,
         24,  87,  94,  25,  14,  39, 101,  89,   4,  96,  21,  28,  38,  81,
         28,   5,  25,  57,  14,  64,  36,  52,  11,  36,   8, 105,  97,  36,
         43,  89, 102,  54,  56,  34,  86,   3,   4,   2,  89,  61,  66,  43,
         94,  41,  28,  21,  28,  13, 109,  83, 118,  99, 101,  10,  65,  17,
         31,   4,  35,  21,  90,   5

In [9]:
from datetime import datetime

currDT = datetime.now()
currStr = currDT.strftime("%Y%m%d-%H%M%S")

fname = 'Prediction_{}.csv'.format(currStr)
df_preds.to_csv(join(OutPath, fname), index=False)

fname = 'Probability_{}.csv'.format(currStr)
df_probs.to_csv(join(OutPath, fname), index=False)